In [ ]:
import numpy as np

def calculate_gini_index_single(Y):
    a, b = np.unique(Y, return_counts = True)
    b = b/b.sum()
    b = b * b
    return 1 - b.sum()

In [ ]:
def calculate_gini_index(Y_subsets):
    l = len(Y_subsets)
    tot = 0
    new = 0
    for i in range(l):
        l1 = len(Y_subsets[i])
        new += l1*calculate_gini_index_single(Y_subsets[i])
        tot += l1
    new /= tot
    return new

In [ ]:
def split_data_set(data_X, data_Y, feature_index, threshold):
    a = data_X[:, feature_index] >= threshold
    b = ~a
    return data_X[b], data_Y[b], data_X[a], data_Y[a]

In [ ]:
def get_best_split(X, Y):
    sh = X.shape
    l = sh[0]
    f = sh[1]
    all_feat_mini = []
    for i in range(f):
        mini = [i,0,0.6]
        arr = np.array([(X[j,i], Y[j]) for j in range(l)], dtype = [('feat',np.float32),('targ',int)])
        arr.sort(order = 'feat')
        lis, cts = np.unique(arr['feat'], return_counts=True)
        for j in range(1,len(cts)):
            cts[j]+=cts[j-1]
        for j in range(1,len(cts)):
            thresh = lis[j]
            gini = calculate_gini_index([arr['targ'][:cts[j-1]],arr['targ'][cts[j-1]:]])
            if gini < mini[2] and thresh != mini[1] :
                mini[2] = gini
                mini[1] = thresh
        all_feat_mini.append(mini)
    all_feat_mini.sort(key = lambda x : x[2])
    return all_feat_mini[0][0], all_feat_mini[0][1]

In [49]:
class Node:
    def __init__(self, predicted_class, depth):
        self.predicted_class = predicted_class
        self.feature_index = None
        self.threshold = None
        self.depth = depth
        self.left = None
        self.right = None

def construct_tree(X, Y, max_depth, min_size, depth):
    #TODO Complete the function implementation. Read the Question text for details
    ft, th = get_best_split(X, Y)
    a, b = np.unique(Y, return_counts = True)
    pred = a[np.argmax(b)]
    node = Node(pred, depth)
    node.feature_index = ft
    node.threshold = th
    depth += 1
    a = X[:, ft] < th
    b = ~a
    #print(X, ft, th, np.hstack([X[a, :ft], X[a, ft:]]), np.hstack([X[b, :ft], X[b, ft:]]), sep = '\n')
    Yl = Y[a]
    a1, b1 = np.unique(Yl, return_counts = True)
    if len(a1) != 0:
        prob = max(b1)/b1.sum()
        pred = a1[np.argmax(b1)]
    else :
        prob = 1
        pred = node.predicted_class
    if depth <= max_depth and sum(a) > min_size and prob < 1:
        node.left = construct_tree(X[a, :], Yl, max_depth, min_size, depth)
    else :
        node.left = Node(pred, depth)
    Yr = Y[b]
    a1, b1 = np.unique(Yr, return_counts = True)
    if len(a1) != 0 :
        prob = max(b1)/b1.sum()
        pred = a1[np.argmax(b1)]
    else:
        prob = 1
        pred = node.predicted_class
    if depth <= max_depth and sum(b) > min_size and prob < 1 :
        node.right = construct_tree(X[b, :], Yr, max_depth, min_size, depth)
    else :
        node.right = Node(pred, depth)
    return node

In [ ]:
def print_tree(node):
    print(f"X{node.feature_index} {node.threshold}")
    if node.left.feature_index != None :
        print_tree(node.left)
    if node.right.feature_index != None :
        print_tree(node.right)

In [ ]:
def travel(node, dat):
    if node.threshold == None :
        a = node.predicted_class
        return a
    else:
        if dat[node.feature_index] < node.threshold :
            a = travel(node.left, dat)
            return a
        else:
            a = travel(node.right, dat)
            return a

In [ ]:
def predict(root, X):
    preds = []
    for i in range(len(X)):
        a = travel(root, X[i])
        preds.append(a)
    return preds

In [ ]:
import pandas as pd
from google.colab import files
uploaded = files.upload()

Saving test.csv to test (1).csv
Saving train.csv to train (1).csv


In [ ]:
import io
train_dat1 = pd.read_csv(io.BytesIO(uploaded['train.csv']))
test_dat1 = pd.read_csv(io.BytesIO(uploaded['test.csv']))

In [ ]:
train_dat1.head(), test_dat1.head()

(   PassengerId  Survived  Pclass     Sex   Age  SibSp  Parch     Fare
 0            1         0       3    male  22.0      1      0   7.2500
 1            2         1       1  female  38.0      1      0  71.2833
 2            3         1       3  female  26.0      0      0   7.9250
 3            4         1       1  female  35.0      1      0  53.1000
 4            5         0       3    male  35.0      0      0   8.0500,
    PassengerId  Survived  Pclass   Sex   Age  SibSp  Parch     Fare
 0          621         0       3  male  27.0      1      0  14.4542
 1          622         1       1  male  42.0      1      0  52.5542
 2          623         1       3  male  20.0      1      1  15.7417
 3          624         0       3  male  21.0      0      0   7.8542
 4          625         0       3  male  21.0      0      0  16.1000)

In [ ]:
target = train_dat1.Survived.copy()
test_target = test_dat1.Survived.copy()
train_dat1.drop(['PassengerId','Survived'],axis = 1, inplace = True)
test_dat1.drop(['PassengerId','Survived'],axis = 1,inplace = True)
target, train_dat1.head(), test_target, train_dat1.head()

(0      0
 1      1
 2      1
 3      1
 4      0
       ..
 615    1
 616    0
 617    0
 618    1
 619    0
 Name: Survived, Length: 620, dtype: int64,
    Pclass     Sex   Age  SibSp  Parch     Fare
 0       3    male  22.0      1      0   7.2500
 1       1  female  38.0      1      0  71.2833
 2       3  female  26.0      0      0   7.9250
 3       1  female  35.0      1      0  53.1000
 4       3    male  35.0      0      0   8.0500,
 0      0
 1      1
 2      1
 3      0
 4      0
       ..
 266    0
 267    1
 268    0
 269    1
 270    0
 Name: Survived, Length: 271, dtype: int64,
    Pclass     Sex   Age  SibSp  Parch     Fare
 0       3    male  22.0      1      0   7.2500
 1       1  female  38.0      1      0  71.2833
 2       3  female  26.0      0      0   7.9250
 3       1  female  35.0      1      0  53.1000
 4       3    male  35.0      0      0   8.0500)

In [ ]:
train_dat1.Age.mean(), train_dat1.Age.median(), train_dat1.Age.std(), test_dat1.Age.mean(), test_dat1.Age.median(), test_dat1.Age.std()

(29.78118609406953,
 28.0,
 14.404997902599808,
 29.520755555555557,
 28.0,
 14.817980416474304)

In [ ]:
train_dat1.Age.fillna(28,inplace = True), test_dat1.Age.fillna(28,inplace = True)
train_dat1.info(), test_dat1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 620 entries, 0 to 619
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Pclass  620 non-null    int64  
 1   Sex     620 non-null    object 
 2   Age     620 non-null    float64
 3   SibSp   620 non-null    int64  
 4   Parch   620 non-null    int64  
 5   Fare    620 non-null    float64
dtypes: float64(2), int64(3), object(1)
memory usage: 29.2+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271 entries, 0 to 270
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Pclass  271 non-null    int64  
 1   Sex     271 non-null    object 
 2   Age     271 non-null    float64
 3   SibSp   271 non-null    int64  
 4   Parch   271 non-null    int64  
 5   Fare    271 non-null    float64
dtypes: float64(2), int64(3), object(1)
memory usage: 12.8+ KB


(None, None)

In [ ]:
train_dat1.Sex.replace(to_replace=['male','female'], value=[1, 0], inplace=True)
test_dat1.Sex.replace(to_replace=['male','female'], value=[1, 0], inplace=True)

In [ ]:
train_dat1.head(), test_dat1.head()

(   Pclass  Sex   Age  SibSp  Parch     Fare
 0       3    1  22.0      1      0   7.2500
 1       1    0  38.0      1      0  71.2833
 2       3    0  26.0      0      0   7.9250
 3       1    0  35.0      1      0  53.1000
 4       3    1  35.0      0      0   8.0500,
    Pclass  Sex   Age  SibSp  Parch     Fare
 0       3    1  27.0      1      0  14.4542
 1       1    1  42.0      1      0  52.5542
 2       3    1  20.0      1      1  15.7417
 3       3    1  21.0      0      0   7.8542
 4       3    1  21.0      0      0  16.1000)

In [ ]:
trainset = train_dat1.to_numpy()
testset = test_dat1.to_numpy()
traintar = target.to_numpy()
testtar = test_target.to_numpy()

In [50]:
maxdep = len(trainset[0])
minsiz = 10
root = construct_tree(trainset, traintar, maxdep, minsiz, 0)

In [55]:
print(f'Test accuracy is {1-(abs(testtar - predict(root, testset)).sum()/len(testtar))}')

Test accuracy is 0.8118081180811808
